In [1]:
import os
import PIL
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.utils import make_grid, save_image
from gradcam.utils import visualize_cam
from gradcam import GradCAM, GradCAMpp
import nets
from lib import get_datatransformer
from arg import get_params
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
model = nets.vggnet("")
model.load_state_dict(torch.load("temp.dict"))
args = vars(get_params())
transformer = get_datatransformer(args,2)

In [3]:
%matplotlib widget
fig = plt.figure(0)
ax1 = fig.add_subplot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [100]:
label = 0;
roa = {0:"0_MI",1:"1_H"}
dataset_path = '/home/ray/TrainingData/419/ap/10/'
dataset_path += roa[label]+"/"
file_list = os.listdir(dataset_path)
img_name= file_list[round(random.uniform(0,len(file_list)))]
pil_img = PIL.Image.open(dataset_path+img_name)
torch_img = transformer['val'](pil_img).to(device)
normed_torch_img = torch_img.unsqueeze(0).to(device)
print(img_name)
_,pred = torch.max(model(normed_torch_img),1)
print(pred == label)
plt.title(img_name+" "+str(pred.item()==label))

17658_ASMI_ISCIN_p6065_ap7.jpg
tensor([False], device='cuda:0')


Text(0.5, 1.0, '17658_ASMI_ISCIN_p6065_ap7.jpg False')

In [99]:
model(normed_torch_img)

tensor([[ 0.6113, -0.6797]], device='cuda:0', grad_fn=<AddmmBackward>)

In [51]:
configs = [
    dict(model_type='vgg', arch=model, layer_name='features_51'),
]
for config in configs:
    config['arch'].to(device).eval()

cams = [
    [cls.from_config(**config) for cls in (GradCAM, GradCAMpp)]
    for config in configs
]

In [55]:
images = []
images2 = []
for gradcam, gradcam_pp in cams:
    mask, _ = gradcam(normed_torch_img)
    heatmap, result = visualize_cam(mask, torch_img)

    mask_pp, _ = gradcam_pp(normed_torch_img)
    heatmap_pp, result_pp = visualize_cam(mask_pp, torch_img)
    
    images.extend([torch_img.cpu(), heatmap, heatmap_pp, result, result_pp])
    images2.extend([torch_img.cpu(), heatmap_pp,result_pp])
    
grid_image = make_grid(images2, nrow=len(images2))
pil_image = transforms.ToPILImage()(grid_image)
plt.imshow(pil_image)
plt.axis('off')
plt.xticks([])
plt.yticks([])

([], [])

tensor([[ 0.5828, -0.6457]], device='cuda:0', grad_fn=<AddmmBackward>)